In [5]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import PolynomialFeatures

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.neural_network import MLPRegressor

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

In [6]:
df = pd.read_csv('IphoneData.csv')
newdf = df.drop(columns=['NAME', 'CO2E'])
features = newdf.to_numpy()
emissions = df['CO2E'].to_numpy()

In [10]:
def modelScore(X, y, model):
    avgSSE = []
    avgMAPE = []
    splits = LeaveOneOut().get_n_splits(X)
    #cross validation using leave one out
    for i, (trainIndex, testIndex) in enumerate(LeaveOneOut().split(X)):
        X_train = X[trainIndex]
        X_test = X[testIndex]
        y_train = y[trainIndex]
        y_test = y[testIndex]
    

        if model == 'linear regression':
            fittedModel = LinearRegression().fit(X_train, y_train)
                
        elif model == 'decision tree':
            fittedModel = tree.DecisionTreeRegressor().fit(X_train, y_train)
                
        elif model == 'random forest':
            y_train = y_train.flatten()
            RF = RandomForestRegressor()#ccp_alpha=2.6, max_samples=.9, max_features='log2', min_samples_split=2,
                      #n_estimators=50, max_depth=17, min_samples_leaf=1, criterion='squared_error')
            fittedModel = RF.fit(X_train, y_train)
            
        elif model == 'support vector regression':
            y_train = y_train.flatten()
            fittedModel = SVR(kernel = 'linear').fit(X_train, y_train)
    
        elif model == 'polynomial regression':
            polyFeatures = PolynomialFeatures(degree=3, interaction_only=True, include_bias=True, order='C')
            polyX_train = polyFeatures.fit_transform(X_train)
            polyX_test = polyFeatures.transform(X_test)
            fittedModel = LinearRegression().fit(polyX_train, y_train)
            X_test = polyX_test

        elif model == 'xgboost':
            xgbModel = xgb.XGBRegressor(learning_rate=0.225, n_estimators=225, booster='gblinear',
                                        reg_alpha=0, reg_lambda=0, eval_metric='auc', 
                                        objective='reg:squaredlogerror', updater='coord_descent', 
                                        feature_selector='thrifty')
            fittedModel = xgbModel.fit(X_train, y_train)
        
        elif model =='lasso':
            lassoModel = Lasso(tol=.0001, max_iter=2000, selection='random', alpha=.15)
            fittedModel = lassoModel.fit(X_train, y_train)

        elif model =="neural network":
            nn = MLPRegressor(alpha=0.001, batch_size=25, hidden_layer_sizes=(10, 10, 10, 10),
             learning_rate_init=0.05, max_iter=10000, random_state=42, tol=1e-1, max_fun=25000,
             solver='lbfgs', activation='identity')
            
            fittedModel = nn.fit(X_train, y_train)          

        y_pred = fittedModel.predict(X_test)
        
        #scores
        avgSSE.append(np.sum((y_test - y_pred) ** 2))
        avgMAPE.append(mean_absolute_percentage_error(y_test, y_pred))

    avgSSE, avgMAPE = np.mean(avgSSE), np.mean(avgMAPE)

   #print results
    print("SSE = " + str(avgSSE))
    print("MAPE = " + str(avgMAPE))

"""alpha=0.1, batch_size=25, hidden_layer_sizes=(10, 10, 10, 10),
             learning_rate_init=0.5, max_iter=12000, random_state=42, tol=1.01, max_fun=15000,
             solver='lbfgs', activation='identity'"""


"alpha=0.1, batch_size=25, hidden_layer_sizes=(10, 10, 10, 10),\n             learning_rate_init=0.5, max_iter=12000, random_state=42, tol=1.01, max_fun=15000,\n             solver='lbfgs', activation='identity'"

In [11]:
modelScore(features, emissions, 'neural network')

SSE = 19.009770446470295
MAPE = 0.046050816668832206


In [ ]:
modelScore(features, emissions, 'linear regression')

SSE = 19.697207782900612
MAPE = 0.04873289978762851


In [ ]:
modelScore(features, emissions, 'decision tree')

SSE = 63.0
MAPE = 0.08228987584647782


In [ ]:
modelScore(features, emissions, 'random forest')

SSE = 40.18330208333333
MAPE = 0.06607144398932861


In [ ]:
modelScore(features, emissions, 'support vector regression')

KeyboardInterrupt: 

In [ ]:
modelScore(features, emissions, 'polynomial regression')

SSE = 6.48902173028647
MAPE = 0.02359235125487665


In [ ]:
modelScore(features, emissions, 'xgboost')

SSE = 19.403605405051774
MAPE = 0.04430870011778093


In [ ]:
modelScore(features, emissions, 'lasso')

SSE = 20.550335818378592
MAPE = 0.047764495816255324
